# Advanced Workflows and Comprehensive Comparisons

This notebook demonstrates advanced workflows and comprehensive comparisons of all available detectors and scorers in Anomsmith.

We'll cover:
1. Comprehensive detector/scorer comparison
2. Ensemble approaches
3. Real-world workflow patterns
4. Best practices and recommendations


In [ ]:
import numpy as np
import pandas as pd
from plotsmith import plot_timeseries
import matplotlib.pyplot as plt

from anomsmith import (
    detect_anomalies, 
    sweep_thresholds, 
    backtest_detector,
    ThresholdRule
)
from anomsmith.primitives.scorers.robust_zscore import RobustZScoreScorer
from anomsmith.primitives.scorers.statistical import ZScoreScorer, IQRScorer
from anomsmith.primitives.detectors.ml import (
    IsolationForestDetector,
    LOFDetector,
    RobustCovarianceDetector
)
from anomsmith.primitives.detectors.pca import PCADetector
from anomsmith.primitives.detectors.change_point import ChangePointDetector

np.random.seed(42)


## Creating Comprehensive Test Data

We'll create complex data that tests various anomaly types.


In [ ]:
def create_comprehensive_data(n: int = 300, contamination: float = 0.1, seed: int = 42):
    """Create comprehensive test data with various anomaly types."""
    np.random.seed(seed)
    
    # Base series with trend and seasonality
    t = np.arange(n)
    trend = 0.01 * t
    seasonal = 2 * np.sin(2 * np.pi * t / 50)
    noise = np.random.randn(n) * 0.5
    y = trend + seasonal + noise
    
    # Inject different types of anomalies
    n_anomalies = int(n * contamination)
    anomaly_indices = np.random.choice(n, n_anomalies, replace=False)
    
    for idx in anomaly_indices:
        anomaly_type = np.random.choice(['spike', 'level_shift', 'contextual'])
        if anomaly_type == 'spike':
            y[idx] += np.random.choice([-1, 1]) * np.random.uniform(4, 8)
        elif anomaly_type == 'level_shift':
            # Create a small level shift
            shift_size = np.random.uniform(2, 4)
            y[idx:min(idx+5, n)] += np.random.choice([-1, 1]) * shift_size
        else:
            # Contextual anomaly
            y[idx] += np.random.choice([-1, 1]) * np.random.uniform(1.5, 3)
    
    # Create ground truth labels
    labels = pd.Series(np.zeros(n), index=pd.date_range("2020-01-01", periods=n, freq="D"))
    labels.iloc[anomaly_indices] = 1
    
    index = pd.date_range("2020-01-01", periods=n, freq="D")
    y_series = pd.Series(y, index=index)
    
    return y_series, labels, anomaly_indices

y, labels, true_anomaly_indices = create_comprehensive_data(n=300, contamination=0.1)
print(f"Created time series with {len(y)} points")
print(f"True anomalies: {labels.sum()}")
print(f"Anomaly rate: {labels.mean():.2%}")


## Comprehensive Detector Comparison

Let's test all available detectors and scorers on the same data.


In [ ]:
# Initialize all detectors and scorers
threshold_rule = ThresholdRule(method="quantile", value=0.9, quantile=0.9)

detectors_scorers = {
    'RobustZScore': RobustZScoreScorer(epsilon=1e-8),
    'ZScore': ZScoreScorer(),
    'IQR': IQRScorer(),
    'IsolationForest': IsolationForestDetector(contamination=0.1, random_state=42),
    'LOF': LOFDetector(n_neighbors=20, contamination=0.1),
    'RobustCovariance': RobustCovarianceDetector(contamination=0.1, random_state=42),
    'ChangePoint': ChangePointDetector(window_size=10, threshold_multiplier=3.0),
}

# Fit and detect with all
results_all = {}
for name, detector_scorer in detectors_scorers.items():
    detector_scorer.fit(y.values)
    results_all[name] = detect_anomalies(y, detector_scorer, threshold_rule)

# Calculate metrics for each
comparison_metrics = {}
for name, result in results_all.items():
    if labels is not None:
        aligned_labels = labels.reindex(result.index, fill_value=0).values
        aligned_labels = (aligned_labels != 0).astype(int)
        pred_labels = result['flag'].values
        
        from anomsmith.workflows.eval.metrics import (
            compute_precision, compute_recall, compute_f1
        )
        
        precision = compute_precision(aligned_labels, pred_labels)
        recall = compute_recall(aligned_labels, pred_labels)
        f1 = compute_f1(aligned_labels, pred_labels)
    else:
        precision = recall = f1 = np.nan
    
    comparison_metrics[name] = {
        'Anomalies Detected': result['flag'].sum(),
        'Anomaly Rate': result['flag'].mean(),
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Mean Score': result['score'].mean(),
        'Std Score': result['score'].std()
    }

comparison_df = pd.DataFrame(comparison_metrics).T
print("Comprehensive Detector/Scorer Comparison:")
print(comparison_df.round(4))


In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Precision, Recall, F1 comparison
metrics_to_plot = ['Precision', 'Recall', 'F1']
for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    values = comparison_df[metric].sort_values(ascending=False)
    colors = plt.cm.viridis(np.linspace(0, 1, len(values)))
    bars = ax.barh(range(len(values)), values.values, color=colors, alpha=0.7)
    ax.set_yticks(range(len(values)))
    ax.set_yticklabels(values.index)
    ax.set_xlabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, values.values)):
        if not np.isnan(val):
            ax.text(val, i, f' {val:.3f}', va='center', fontsize=10)

# Anomaly detection count
ax = axes[1, 1]
detected_counts = comparison_df['Anomalies Detected'].sort_values(ascending=False)
colors = plt.cm.plasma(np.linspace(0, 1, len(detected_counts)))
bars = ax.barh(range(len(detected_counts)), detected_counts.values, color=colors, alpha=0.7)
ax.set_yticks(range(len(detected_counts)))
ax.set_yticklabels(detected_counts.index)
ax.set_xlabel('Anomalies Detected', fontsize=12)
ax.set_title('Detection Count Comparison', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
ax.axvline(labels.sum(), color='r', linestyle='--', linewidth=2, 
          label=f'True Anomalies ({int(labels.sum())})')
ax.legend()

plt.tight_layout()
plt.show()


## Best Practices and Recommendations

### When to Use Each Detector/Scorer:

1. **Statistical Scorers (ZScore, IQR, RobustZScore)**
   - Fast and interpretable
   - Good for univariate time series
   - RobustZScore is most robust to outliers

2. **Machine Learning Detectors (Isolation Forest, LOF, Robust Covariance)**
   - Better for complex patterns
   - Isolation Forest: Fast, good for high-dimensional data
   - LOF: Good for local anomalies
   - Robust Covariance: Good for multivariate Gaussian data

3. **PCA Detector**
   - Excellent for multivariate data
   - Captures main patterns in normal data

4. **Change Point Detector**
   - Best for detecting structural breaks
   - Good for regime changes and level shifts

### Workflow Recommendations:

1. Start with simple statistical scorers
2. Use threshold sweeping to find optimal thresholds
3. Validate with backtesting
4. Consider ensemble approaches for critical applications
5. Monitor performance over time


## Summary

In this comprehensive notebook, we've:
1. Compared all available detectors and scorers
2. Evaluated performance using multiple metrics
3. Visualized comprehensive comparisons
4. Provided best practices and recommendations

Key takeaways:
- Different detectors excel at different anomaly types
- Statistical methods are fast and interpretable
- ML methods handle complex patterns better
- Always validate with backtesting
- Consider your specific use case when choosing detectors
